In [0]:
pip install tweepy==3.10.0

In [0]:
from tweepy import OAuthHandler, API, StreamListener, Stream
from json import loads, dumps
from datetime import datetime
 
# Filtros de pesquisa
search = [""]
 
# Tokens de acesso a API
ACCESS_TOKEN = "<ACCESS_TOKEN>"
ACCESS_TOKEN_SECRET = "<ACCESS_TOKEN_SECRET>"
CONSUMER_KEY = "<CONSUMER_KEY>"
CONSUMER_SECRET = "<CONSUMER_SECRET>"
 
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth) 
 
print('Adicionando e salvando os Tweets...')
 
class Output():
    def __init__(self):
        self.tweet_dates = [] # Criando Lista que salva as datas de cada Tweet     
        self.output = [] # Criando Lista que salva os tweets
        self.limit, self.count = 100, 0 # Criando contador e limite para o número de arquivos que devem ser salvos em cada JSON
        
    def update(self, tweet):
        self.tweet_dates.append(datetime.fromtimestamp(int(tweet["timestamp_ms"])/1000)) # Selecionando o coluna "timestamp_ms", fazendo conversões e adicionando-o a lista "tweet_dates"
        self.output.append(tweet) # Adicionando um tweet a lista "output"
        self.count += 1
        if self.count == self.limit: # Quando o contador chegar ao limite pré-definido é chamado a função "save"
            self.save()
            
    def save(self): 
        self.count = 0 # Reinicia o contador a uma unidade
        date = sorted(self.tweet_dates)[0].strftime("%Y-%m-%d %H:%M:%S") # Pega a primeira ocorrência de data e cria uma mascara a ela
        print("Tweet salvo...")
        
    #Salvando os arquivos na pasta do DataLake Storage Gen2 e limpando as listas "output" e "tweet_dates"
        with open(f'<LOCAL_PATH>{date}.json', 'w') as file:
            file.write(dumps(self.output))
        self.output.clear() # Limpa a lista "Output"
        self.tweet_dates.clear()  # Limpa a lista "tweet_dates"
        
class MyStreamListener(StreamListener):
    output = Output()
    def on_data(self, data):
        self.output.update(loads(data))
        
listener = MyStreamListener()
stream = Stream(auth, listener)
stream.filter(languages=["pt"], track = search)